In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 安裝必要套件
!pip install transformers
!pip install nltk

# 匯入函式庫
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

from transformers import get_scheduler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 載入資料
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/sample_submission.csv")

# 使用 text 欄位與 target 作為輸入與標籤
train_texts = train['text'].astype(str).tolist()
train_labels = train['target'].tolist()
test_texts = test['text'].astype(str).tolist()

# 使用 BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 定義 Dataset 類別
class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

# 分割驗證集
train_texts_split, val_texts, train_labels_split, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=42)

# 建立 dataset
train_dataset = TweetDataset(train_texts_split, train_labels_split)
val_dataset = TweetDataset(val_texts, val_labels)
test_dataset = TweetDataset(test_texts)

# 建立 dataloader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# 載入 BERT 模型
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# 設定 optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# 設定 scheduler
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0,
                             num_training_steps=num_training_steps)

# 訓練模型
model.train()
for epoch in range(3):
    print(f"Epoch {epoch + 1}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

# 預測 test set
model.eval()
predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# 生成 submission 檔案
submission['target'] = predictions
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/data/submission.csv", index=False)
print("✅ submission.csv 已儲存完成")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1


Epoch 0: 100%|██████████| 429/429 [01:41<00:00,  4.24it/s, loss=0.2]


Epoch 2


Epoch 1: 100%|██████████| 429/429 [01:41<00:00,  4.25it/s, loss=0.0768]


Epoch 3


100%|██████████| 204/204 [00:13<00:00, 15.54it/s]

✅ submission.csv 已儲存完成
